# Leer datos

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/items_titles.csv')

df

KeyboardInterrupt: 

In [ ]:
import unicodedata

# Función para limpiar el texto
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar acentos
    text = ''.join(
        char for char in unicodedata.normalize('NFKD', text) if not unicodedata.combining(char)
    )
    return text

# Aplicar la limpieza a la columna 'ITE_ITEM_TITLE'
df['cleaned_title'] = df['ITE_ITEM_TITLE'].apply(preprocess_text)

# Mostrar los primeros resultados
print("Primeros registros después de la limpieza:")
print(df[['ITE_ITEM_TITLE', 'cleaned_title']].head())

# Tokenización.

In [ ]:
import re

# Función para tokenizar con eliminación de puntuación
def tokenize_with_re(text):
    # Dividir en palabras ignorando puntuación
    return re.findall(r'\b\w+\b', text)

# Aplicar tokenización con re
df['tokens'] = df['cleaned_title'].apply(tokenize_with_re)

# Mostrar los primeros resultados
print("Primeros registros después de la tokenización:")
print(df[['cleaned_title', 'tokens']].head())

Se identifica que los productos están en portugués.

In [ ]:
from collections import Counter

# Unir todas las palabras en una lista
all_words = [word for tokens in df['tokens'] for word in tokens]

# Contar la frecuencia de cada palabra
word_counts = Counter(all_words)

# Obtener las 10 palabras más frecuentes
most_common_words = word_counts.most_common(10)

print("Palabras más frecuentes:")
print(most_common_words)

In [ ]:
from nltk import ngrams

# Función para generar n-gramas
def generate_ngrams(tokens, n):
    return list(ngrams(tokens, n))

# Crear n-gramas (bi-gramas y tri-gramas)
df['bigrams'] = df['tokens'].apply(lambda x: generate_ngrams(x, 2))
df['trigrams'] = df['tokens'].apply(lambda x: generate_ngrams(x, 3))

# Contar los bigramas más comunes
all_bigrams = [bigram for bigrams in df['bigrams'] for bigram in bigrams]
bigram_counts = Counter(all_bigrams)

# Contar los trigramas más comunes
all_trigrams = [trigram for trigrams in df['trigrams'] for trigram in trigrams]
trigram_counts = Counter(all_trigrams)

print("Bigrams más frecuentes:")
print(bigram_counts.most_common(10))

print("Trigrams más frecuentes:")
print(trigram_counts.most_common(10))

In [ ]:
import matplotlib.pyplot as plt

# Longitud de cada título
df['num_tokens'] = df['tokens'].apply(len)

# Estadísticas descriptivas
mean_tokens = df['num_tokens'].mean()
max_tokens = df['num_tokens'].max()
min_tokens = df['num_tokens'].min()

print(f"Longitud promedio de tokens por título: {mean_tokens:.2f}")
print(f"Longitud máxima: {max_tokens}")
print(f"Longitud mínima: {min_tokens}")

# Graficar la distribución de la longitud
plt.figure(figsize=(10, 6))
plt.hist(df['num_tokens'], bins=20, color='skyblue', edgecolor='black')
plt.title("Distribución de la longitud de tokens por título")
plt.xlabel("Número de tokens")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir los tokens a texto limpio nuevamente
df['cleaned_text'] = df['tokens'].apply(lambda x: ' '.join(x))

# Crear la matriz TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned_text'])

# Obtener palabras clave con TF-IDF más alto para el primer título
feature_names = vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix[0].T.todense().tolist()
keywords = [(feature_names[i], score[0]) for i, score in enumerate(tfidf_scores) if score[0] > 0]
keywords = sorted(keywords, key=lambda x: x[1], reverse=True)

print("Palabras clave para el primer título según TF-IDF:")
print(keywords[:10])

### Análisis de Títulos:

1. **Palabras más frecuentes**:
   - Identificar términos comunes como `tenis` (22,600 apariciones) o `feminino` (7,931) nos permite detectar patrones generales en los productos.
   - Estas palabras frecuentes pueden no ser útiles directamente para medir similitud, ya que están presentes en muchos títulos, pero ayudan a filtrar palabras no relevantes.

2. **Bigramas y Trigramas más frecuentes**:
   - Frases clave como `tenis feminino casual` y `bicicleta aro 29` destacan categorías y características específicas de los productos.
   - Estas combinaciones pueden ser valiosas para crear representaciones más específicas de los títulos y mejorar la detección de similitudes.

3. **Estadísticas descriptivas**:
   - La longitud promedio de 7.26 tokens por título indica que los textos son concisos, lo que favorece la eficiencia al calcular similitudes.
   - Las longitudes máxima y mínima (28 y 1 tokens, respectivamente) sugieren la necesidad de manejar casos extremos, como títulos muy cortos o detallados, en la comparación.

4. **Palabras clave según TF-IDF**:
   - Términos con altos puntajes como `posh` y `ascension` indican elementos únicos que pueden ser cruciales para identificar productos similares.
   - Palabras con puntajes bajos (`tenis`, `masculino`) son menos útiles para diferenciar productos.

**Conclusión (En el contexto del reto)**:
- Este análisis permite priorizar características clave de los títulos (e.g., bigramas y trigramas únicos) para calcular similitudes entre productos.
- El uso de TF-IDF es una opción viable para representar títulos y calcular similitudes basadas en coseno o distancias.
- La eficiencia en tiempo de ejecución dependerá de optimizaciones en el algoritmo y la elección de estructuras de datos escalables.

# Eliminar Stop-words.

In [ ]:
from nltk.corpus import stopwords
import nltk

# Descargar stop words si no están disponibles
nltk.download('stopwords')

# Combinar stop words en portugués
stop_words_pt = set(stopwords.words('portuguese'))


# Función para eliminar stop words de múltiples idiomas
def remove_multilang_stopwords(tokens):
    return [word for word in tokens if word not in stop_words_pt]

# Aplicar la función a los tokens
df['filtered_tokens'] = df['tokens'].apply(remove_multilang_stopwords)

# Verificar resultados
print("Primeros registros sin stop words:")
print(df[['tokens', 'filtered_tokens']].head())

In [ ]:
# Comparar longitud antes y después de eliminar stop words
df['original_length'] = df['tokens'].apply(len)
df['filtered_length'] = df['filtered_tokens'].apply(len)

# Calcular palabras eliminadas
df['removed_words'] = df['original_length'] - df['filtered_length']

# Estadísticas descriptivas
average_removed = df['removed_words'].mean()
percentage_reduction = (average_removed / df['original_length'].mean()) * 100

# Mostrar resultados
print(f"Promedio de palabras eliminadas: {average_removed:.2f}")
print(f"Porcentaje de reducción en el tamaño de los tokens: {percentage_reduction:.2f}%")

# Distribución de palabras eliminadas
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.hist(df['removed_words'], bins=20, color='skyblue', edgecolor='black')
plt.title("Distribución de palabras eliminadas por título")
plt.xlabel("Número de palabras eliminadas")
plt.ylabel("Frecuencia")
plt.show()

### Justificación para la eliminación de Stop Words

1. **Reducción de ruido semántico**:
   - Las stop words, como "de", "para", "e" (en portugués), son palabras comunes que no aportan información significativa para identificar similitudes entre productos.
   - Su presencia puede generar relaciones artificiales entre títulos que comparten conectores pero no son realmente similares.

2. **Foco en términos clave**:
   - Al eliminar las stop words, el análisis se concentra en palabras relevantes como nombres de productos, categorías, y características distintivas.
   - Esto mejora la precisión al identificar títulos con contenido semántico significativo.

3. **Minimización de similitudes irreales**:
   - Títulos como `"Tenis de homem"` y `"Bicicleta para mulher"` comparten la palabra "para", pero no tienen relación real. Al eliminar este tipo de palabras, evitamos asociaciones incorrectas.

4. **Impacto controlado**:
   - Aunque la eliminación de stop words reduce el tamaño de los tokens en solo un **5.18%**, elimina un promedio de **0.38 palabras por título**, lo que ayuda a reducir ruido sin afectar significativamente el contenido útil.

**Conclusión**:
- La eliminación de stop words es una estrategia clave para mejorar la precisión en la identificación de similitudes entre productos, al enfocarse en términos que realmente definen la relación entre ellos.

In [ ]:
# Filtrar títulos que contengan "para" o "de"
filtered_examples = df[df['tokens'].apply(lambda x: 'para' in x or 'de' in x)]

# Mostrar algunos ejemplos
print("Ejemplos de títulos con 'para' o 'de':")
print(filtered_examples[['ITE_ITEM_TITLE', 'tokens']].head(10))

### Validación de la hipótesis: Minimización de similitudes irreales

**Ejemplos analizados**:
- Títulos con palabras comunes como "para" o "de" incluyen:
  - `"Tenis Para Caminhada Super Levinho Spider Corrida"`
  - `"Sapatilha Bike Absolute Nero Mtb Para Pedal Clip"`
  - `"Tenis Feminino De Fazer Caminhada Corrida Academia"`
  - `"Tenis Sneacker Feminino Surfista Cheia De Marca"`
  - `"Tenis De Menino Bota Com Luz De Led Sapato Preto"`

**Observaciones**:
1. **Patrón recurrente de conectores comunes**:
   - Palabras como "para" y "de" aparecen en múltiples títulos, sin aportar información semántica relevante.
   - Ejemplo: `"Tenis Para Caminhada"` y `"Bicicleta Para Mulher"` comparten "para", pero representan productos completamente diferentes.

2. **Ruido en la similitud**:
   - La presencia de estas palabras puede llevar a similitudes artificiales entre títulos que no tienen una relación real, al estar basadas únicamente en palabras comunes.

**Conclusión**:
Eliminar palabras comunes como "para" y "de" mejora la precisión al calcular similitudes entre productos, al reducir el ruido semántico y enfocar el análisis en términos clave.

# Vectorización, similitud usando TF-IDF 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Unir tokens en un solo texto para cada título
df['filtered_text'] = df['filtered_tokens'].apply(lambda x: ' '.join(x))

# Crear la matriz TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['filtered_text'])

print("Matriz TF-IDF generada con forma:", tfidf_matrix.shape)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
from joblib import Parallel, delayed

def calculate_similarity(pair, cosine_sim, df):
    """
    Calcula la similitud de coseno para un par de índices.

    Args:
        pair (tuple): Tupla con los índices de los textos a comparar.
        cosine_sim (ndarray): Matriz de similitud de coseno.
        df (pd.DataFrame): DataFrame original con los textos.

    Returns:
        list: Lista con los títulos y su puntaje de similitud.
    """
    i, j = pair
    title1 = df.iloc[i]['ITE_ITEM_TITLE']
    title2 = df.iloc[j]['ITE_ITEM_TITLE']
    score = cosine_sim[i, j]
    return [title1, title2, score]

# Calcular la matriz de similitud
cosine_sim = cosine_similarity(tfidf_matrix)

# Crear combinaciones únicas de índices
pairs = list(combinations(range(len(df)), 2))

# Paralelizar el cálculo de similitudes
results = Parallel(n_jobs=-1)(delayed(calculate_similarity)(pair, cosine_sim, df) for pair in pairs)

# Crear el DataFrame final con los resultados
similarity_df = pd.DataFrame(results, columns=['ITE_ITEM_TITLE_1', 'ITE_ITEM_TITLE_2', 'Score Similitud'])

# Ordenar por el puntaje de similitud en orden descendente
similarity_df = similarity_df.sort_values(by='Score Similitud', ascending=False)

# Mostrar los primeros resultados
print(similarity_df.head())

# Guardar los resultados en un archivo.
similarity_df.to_csv('../data/output/similarities_output.csv', index=False)

In [ ]:
import time

# Función para medir el tiempo de ejecución
def measure_execution_time(tfidf_matrix):
    start_time = time.time()
    cosine_similarity(tfidf_matrix)
    end_time = time.time()
    return end_time - start_time

# Evaluar en diferentes tamaños del dataset
sizes = [100, 500, 1000, len(df)]
times = []

for size in sizes:
    sample_matrix = tfidf_matrix[:size, :]
    exec_time = measure_execution_time(sample_matrix)
    times.append((size, exec_time))

# Mostrar los tiempos
print("Tiempos de ejecución:")
for size, exec_time in times:
    print(f"Tamaño: {size} - Tiempo: {exec_time:.4f} segundos")

In [ ]:
from scripts/similarity import TextSimilarityPipeline

# Crear instancia y ejecutar
similitud = TextSimilarityPipeline(df)
resultado = similitud.run_pipeline()

# Mostrar resultado
resultado